In [ ]:
import numpy as np
from matplotlib import pylab as plt
import matplotlib.cm as cm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
import pandas as pd
# import warnings
# warnings.filterwarnings("ignore")
%matplotlib inline

Read CSV file with words and assigned emotion

In [ ]:
raw_emotions = pd.read_csv('emotion-lexicon.csv')
raw_emotions.shape

(14182, 11)

In [ ]:
raw_emotions.head()
# focus on joy, sadness, fear, disgust, anger, surprise

,English (en),Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,aback,0,0,0,0,0,0,0,0,0,0
1,abacus,0,0,0,0,0,0,0,0,0,1
2,abandon,0,1,0,0,0,1,0,1,0,0
3,abandoned,0,1,1,0,0,1,0,1,0,0
4,abandonment,0,1,1,0,0,1,0,1,1,0


In [ ]:
joy_words = set(raw_emotions.groupby('Joy').agg(list).reset_index()['English (en)'][1])
print(f'Number of Joy Words: {len(joy_words)}')
sad_words = set(raw_emotions.groupby('Sadness').agg(list).reset_index()['English (en)'][1])
print(f'Number of Sad Words: {len(sad_words)}')
fear_words = set(raw_emotions.groupby('Fear').agg(list).reset_index()['English (en)'][1])
print(f'Number of Fear Words: {len(fear_words)}')
disgust_words = set(raw_emotions.groupby('Disgust').agg(list).reset_index()['English (en)'][1])
print(f'Number of Disgust Words: {len(disgust_words)}')
anger_words = set(raw_emotions.groupby('Anger').agg(list).reset_index()['English (en)'][1])
print(f'Number of Anger Words: {len(anger_words)}')
surprise_words = set(raw_emotions.groupby('Surprise').agg(list).reset_index()['English (en)'][1])
print(f'Number of Surprise Words: {len(surprise_words)}')

Number of Joy Words: 689
Number of Sad Words: 1191
Number of Fear Words: 1476
Number of Disgust Words: 1058
Number of Anger Words: 1247
Number of Surprise Words: 534


In [ ]:
raw_songs = pd.read_csv('spotify_songs.csv')
raw_songs.shape

FileNotFoundError: ignored

In [ ]:
raw_songs.head()

In [ ]:
# only look at english songs
raw_songs = raw_songs.loc[raw_songs['language'] == 'en']
# add label col
# raw_songs['label'] = ''
raw_songs.shape

In [ ]:
# for each song, count the number of words of each emotion and label it the emotion with the highest count
emotion_words = {'Joy': joy_words, 'Sadness': sad_words, 'Fear': fear_words, 'Disgust': disgust_words, 'Anger': anger_words, 'Surprise': surprise_words}
labels = np.array([])
def get_emotion_counts(song, emotion_words, labels):
  emotion_counts = {'Joy': 0, 'Sadness': 0, 'Fear': 0, 'Disgust': 0, 'Anger': 0, 'Surprise': 0}
  for emotion, count in emotion_counts.items():
    for word in emotion_words[emotion]:
      if word in song['lyrics']:
        emotion_counts[emotion] += 1
  # print('label = ', max(emotion_counts, key=emotion_counts.get))
  label = max(emotion_counts, key=emotion_counts.get)
  # labels = np.append(labels, label)
  # print(song['label'])
  return label

r = raw_songs.apply(lambda row: get_emotion_counts(row, emotion_words, labels), axis=1)
# raw_songs['label'] = labels

In [ ]:
raw_songs['label'] = r


In [ ]:
raw_songs.groupby('label')['label'].value_counts()

In [ ]:
# from google.colab import drive
# drive.mount('drive')

In [ ]:
# raw_songs.to_csv('labeled_songs.csv')

In [ ]:
happy_songs = raw_songs.loc[raw_songs['label'] == 'Joy']
sad_songs = raw_songs.loc[raw_songs['label'] == 'Sadness']
fear_songs = raw_songs.loc[raw_songs['label'] == 'Fear']
anger_songs = raw_songs.loc[raw_songs['label'] == 'Anger']
disgust_songs = raw_songs.loc[raw_songs['label'] == 'Disgust']
surprise_songs = raw_songs.loc[raw_songs['label'] == 'Surprise']

In [ ]:
import numpy as np
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from matplotlib import pylab as plt
import matplotlib.cm as cm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
import pandas as pd
%matplotlib inline

In [ ]:
# pulled from main notebook
# 12/9: Trying out creating BoW model on list of happy words instead of lyrics
#reading in dataset created from spotify 
spotify_songs = pd.read_csv('all_songs_1126_noNONE_allEN.csv')

# vectorizer = CountVectorizer(binary=True, min_df=1)
# X_happy = vectorizer.fit_transform(joy_words)

# print('Size of Vocabulary/Features:', X_happy.shape)
# print('vectorizer:', vectorizer.get_feature_names())


In [ ]:
kaggle_data = raw_kaggle_songs.sample(n=6000, random_state=1).reset_index(drop=True)
kaggle_data['isHappy'] = kaggle_data.apply(lambda row: 1 if row['label'] == 'Joy' else 0, axis=1)
kaggle_data['isSad'] = kaggle_data.apply(lambda row: 1 if row['label'] == 'Sadness' else 0, axis=1)
kaggle_data = kaggle_data.drop_duplicates()


In [ ]:
from sklearn_pandas import DataFrameMapper
kaggle_data_combo = kaggle_data.copy()
kaggle_data_combo['keywords'] = kaggle_data_combo['lyrics']
# add happy words to each row
# joy = ' '.join(word for word in joy_words)
# sad = ' '.join(word for word in sad_words)
# spotify_songs['keywords'] = [joy] * len(spotify_songs)
# update sad songs with the sad words
spotify_songs['keywords'] = spotify_songs.apply(lambda row: joy_words if row['isHappy'] == 1 else sad_words, axis=1)

x_combo_set = spotify_songs[['danceability', 'energy', 'loudness', 'valence', 'keywords']]
y_combo_set = spotify_songs['isHappy']

# mapper = DataFrameMapper([
#     (['danceability', 'energy', 'loudness', 'valence'], None),
#     ('keywords', CountVectorizer(binary=True, min_df=1))
# ])
# combined_X = mapper.fit_transform(x_combo_set)
# print('combined_X shape:', combined_X.shape)
combo_model = LogisticRegression(penalty='l2').fit(x_combo_set, y_combo_set)


TypeError: ignored

In [ ]:
spotify_songs

,Unnamed: 0,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,isHappy,keywords
0,0,Ed Sheeran,Shivers,Shivers,75MNhvTCCKsST3YqqUiU9r,0.788,0.859,2,-2.724,1,0.0856,0.000000,0.0424,0.822,141.020,207853,4,I took an arrow to the heart\nI never kissed a...,1,meditate aspiring progress succeeding fancy cl...
1,1,Oleta Adams,Evolution,Window Of Hope,7IyXO4OsPqqmB0v1ebT9Ok,0.619,0.820,0,-8.016,1,0.0309,0.000000,0.3020,0.838,111.010,262893,4,Verse 1:\nIn the darkness of your private room...,1,meditate aspiring progress succeeding fancy cl...
2,2,Ph.D.,I Won't Let You Down,I Won't Let You Down,1NXHwS75sCmkMZAE0zvwl0,0.553,0.406,9,-10.498,1,0.0280,0.686000,0.1250,0.231,108.713,242535,4,You ask me if I'm happy here\nNo doubt about i...,1,meditate aspiring progress succeeding fancy cl...
3,3,Chef'Special,Amigo,Amigo,6lAqwGwFzQ44SrODqr0NmQ,0.559,0.922,6,-4.597,1,0.0504,0.000000,0.3560,0.638,145.000,180133,4,You don’t know me I don’t know who you are\nBu...,1,meditate aspiring progress succeeding fancy cl...
4,4,Ed Sheeran,Bad Habits,Bad Habits,6PQ88X9TkUIAUIZJHW2upE,0.808,0.897,11,-3.712,0,0.0348,0.000031,0.3640,0.591,126.026,231041,4,"(One, two, three, four)\nOoh, ooh\n\nEvery tim...",1,meditate aspiring progress succeeding fancy cl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5681,5685,Alvvays,Antisocialites,Dreams Tonite,0DRvClQbN9znQ295sx76VC,0.511,0.622,2,-5.727,1,0.0242,0.000005,0.1270,0.425,99.934,196027,4,"Rode here on the bus, now you're one of us\nIt...",0,confinement epitaph anthrax apathetic cumberso...
5682,5686,Phosphorescent,Pride,Wolves,5IPIpXq8znsSOybCsbd88v,0.331,0.288,1,-11.067,1,0.0311,0.004530,0.0756,0.097,130.752,374640,3,"Mama, there's wolves in the house\nMama, they ...",0,confinement epitaph anthrax apathetic cumberso...
5683,5687,Rex Orange County,Loving Is Easy,Loving Is Easy,5EYi2rH4LYs6M21ZLOyQTx,0.756,0.609,3,-7.927,1,0.0544,0.004740,0.0960,0.537,117.023,155720,4,Loving is easy\nYou had me fucked up\nIt used ...,0,confinement epitaph anthrax apathetic cumberso...
5684,5688,Youth Lagoon,The Year of Hibernation,17,7GDbkrFBGrCb8ntbQPY3hQ,0.352,0.254,0,-8.510,1,0.0473,0.580000,0.1080,0.102,79.989,236000,4,Roaming the campground out by the lake where w...,0,confinement epitaph anthrax apathetic cumberso...


In [ ]:
kaggle_data_combo
combined_X_spotify = mapper.transform(kaggle_data_combo[['danceability', 'energy', 'loudness', 'valence', 'keywords']])
print('combined_X_kaggle shape:', combined_X_spotify.shape)

combo_preds = combo_model.predict(combined_X_spotify)
kaggle_data_combo['combo_features_preds'] = combo_preds
kaggle_data_combo

combined_X_kaggle shape: (6000, 1845)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Unnamed: 0,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language,label,isHappy,isSad,keywords,combo_features_preds
0,8772,3rLZh34oJ1KzLkYwPVYUgP,Anywhere - Willy William Remix,Rita Ora,Time flies by when the night is young Daylight...,27,7JdaTcp2GIiBv3XSGLAe8W,Anywhere (Willy William Remix),2017-11-24,EDM TROPICAL,3pS63EDS40FVGYL41zAcU4,latin,tropical,0.715,0.556,11,-10.456,0,0.0323,0.00283,0.002210,0.1860,0.119,106.970,213084,en,Sadness,0,1,Time flies by when the night is young Daylight...,1
1,8821,3ScJy88F8KqGDfWu8XJhHx,If I Lose Myself - Alesso vs OneRepublic,OneRepublic,I stared up at the sun Thought of all of the p...,67,42UJjk8i8L0De7lQtu7sqi,Native,2014-01-01,Electropop Hits 2017-2020,7kyvBmlc1uSqsTL0EuNLrx,pop,electropop,0.519,0.754,2,-5.689,1,0.0402,0.25800,0.000004,0.1390,0.160,125.975,215187,en,Fear,0,0,I stared up at the sun Thought of all of the p...,1
2,1730,0RZXNlGPvfMo54fd6uCvAT,Knuck If You Buck (feat. Lil' Scrappy),Crime Mob,"Knuck if you buck, boy (Crime Mob, hoe) Knuck ...",60,09stXr7AeoB1PsE3RpMpyU,Crime Mob (U.S. PA Version),2004-07-13,Hip-Hop Drive,37i9dQZF1DWUFmyho2wkQU,rap,hip hop,0.865,0.844,8,-4.946,1,0.3110,0.05820,0.000000,0.3730,0.755,150.090,205120,en,Anger,0,0,"Knuck if you buck, boy (Crime Mob, hoe) Knuck ...",1
3,7881,3fCqWsYPCPdLteUpeIOeyQ,Let's Wait Awhile,Janet Jackson,There's something I want to tell you There's s...,1,3W2rEBPIwXyogofa5b4e6x,Control,1986-02-04,New Jack Swing,3ykXidKLz1eYPvuGoFlD1e,r&b,new jack swing,0.592,0.188,1,-17.243,1,0.0377,0.82000,0.000000,0.1320,0.378,172.683,276689,en,Joy,1,0,There's something I want to tell you There's s...,1
4,7142,31QRCKy7JIwPBaQl9ArGBz,Who's Laughing Now,Jessie J,"Mummy, they call me names, they wouldn't let m...",49,0BZbTNqpXFg6lxNv78X7Lp,Who You Are (Platinum Edition),2011-01-01,post teen pop,6rjxP7GQKoqqgoakzxl3PY,pop,post-teen pop,0.784,0.842,1,-4.041,1,0.1870,0.00448,0.000029,0.1040,0.483,100.035,234880,en,Sadness,0,1,"Mummy, they call me names, they wouldn't let m...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,144,04g1aQFzaegB7ovWaMMIzz,Congratulations - Don Diablo VIP Mix,Don Diablo,Dreaming Dreaming about the days that we could...,51,4PFXME1mmhj54RxWMxH5UX,Congratulations (Don Diablo VIP Mix),2020-01-17,Electro House Top Tracks,1G0q0NK7g3C0XerNqq7GbL,edm,electro house,0.818,0.879,7,-4.677,0,0.2050,0.09120,0.005610,0.1400,0.256,123.980,195484,en,Sadness,0,1,Dreaming Dreaming about the days that we could...,1
5996,5971,2LBqCSwhJGcFQeTHMVGwy3,Die For You,The Weeknd,I'm findin' ways to articulate the feeling I'm...,67,4AdZV63ycxFLF6Hcol0QnB,Starboy,2016-11-24,Today's Hits (Clean),7ENISpOJhocpMJVcGb0qcT,r&b,hip pop,0.586,0.525,1,-7.163,0,0.0615,0.11100,0.000000,0.1340,0.508,133.629,260253,en,Sadness,0,1,I'm findin' ways to articulate the feeling I'm...,1
5997,10551,4jJEQg6mr3fQsUCYTwhBEZ,Sun,Two Door Cinema Club,Ocean blue What have I done to you? Cut so dee...,60,6CPwf8Yo8a4cgBDVeShL5C,Beacon,2012-09-03,Indie/Jazz Poptimism,6OaTudLqBEuTyUMPxRNpZc,pop,indie poptimism,0.647,0.534,4,-5.439,0,0.0279,0.01100,0.000249,0.1040,0.673,96.980,187720,en,Sadness,0,1,Ocean blue What have I done to you? Cut so dee...,1
5998,11187,4rFKp5rDJS3bldnga3QDLY,Everything and More (feat. Aaron Cole),Hollyn,I was wrong (Wrong) to ever let you go You're ...,43,5RoaRB24uoKbClziJyISOo,Everything and More / Isaac,2018-10-19,Christian Dance Party,37i9dQZF1DWUUPO0Sbx2CM,pop,dance pop,0.656,0.700,4,-7.176,0,0.0431,0.24900,0.000000,0.2210,0.293,100.010,197000,en,Fear,0,0,I was wrong (Wrong) to ever let you go You're ...,1
